# Tema 6 - Practica 1: Redis

In [4]:
!pip install redis
import redis
import json


# Apartado A

Configura la conexión a la BBDD Redis que previamente has levantado en Docker.

In [5]:
# APARTADO A

# Conexión a la BBDD Redis
# decode_responses=True --> para que nos devuelva strings en lugar de bytes
try:
    r = redis.Redis(host='localhost', port=6379, decode_responses=True)
    r.ping()
    print("¡Conexión Exitosa a Redis!")
except redis.exceptions.ConnectionError as e:
    print(f"Error al conectar a Redis: {e}")

# Comprobamos que la BBDD está vacía
keys = r.keys('*')
print(f"La BBDD está vacía. Nº de claves: {len(keys)}")

¡Conexión Exitosa a Redis!
La BBDD está vacía. Nº de claves: 0


# Apartado B

Configura un metodo "crear_carrito" que cree un carrito de la compra con la cesta vacía, un username vinculado y un importe total de 0.

In [18]:
def crear_carrito(username):
    """
    Crea un carrito de compra vacío para un usuario.
    
    Args:
        username (str): Nombre del usuario
    
    Returns:
        dict: Carrito inicializado con cesta vacía e importe total 0
    """
    carrito = {
        "username": username,
        "cesta": [],
        "importe_total": 0
    }
    
    # Guardo el carrito en Redis usando el username como clave
    r.set(f"carrito:{username}", json.dumps(carrito))
    
    print(f"Carrito creado para el usuario '{username}'")
    return carrito

In [19]:
# Pruebo la función
mi_carrito = crear_carrito("maria_lopez")
print(f"\n✅ Resultado: {mi_carrito}")

Carrito creado para el usuario 'maria_lopez'

✅ Resultado: {'username': 'maria_lopez', 'cesta': [], 'importe_total': 0}


# Apartado C

Crear una funcion que permita recuperar el contenido del carrito

In [16]:
def recuperar_carrito(username):
    """
    Recupera el carrito de un usuario desde Redis.
    """
    carrito_json = r.get(f"carrito:{username}")
    
    if carrito_json:
        carrito = json.loads(carrito_json)
        print(f"Carrito recuperado para '{username}'")
        return carrito
    else:
        print(f"No existe un carrito para el usuario '{username}'")
        return None

In [17]:
# Pruebo recuperar el carrito
carrito_recuperado = recuperar_carrito("maria_lopez")
print(f"✅ Carrito: {carrito_recuperado}")

Carrito recuperado para 'maria_lopez'
✅ Carrito: {'username': 'maria_lopez', 'cesta': [], 'importe_total': 0}


# Apartado D

Funcion que calcule el importe total dada una cesta de la compra

In [21]:
def calcular_total(cesta):
    """
    Calcula el importe total de una cesta.
    """
    total = 0
    for item in cesta:
        total += item['cantidad'] * item['precio_unitario']
    return total

In [22]:
# Pruebo calcular total
cesta_test = [
    {"articulo": "Manzanas", "cantidad": 3, "precio_unitario": 1.5},
    {"articulo": "Pan", "cantidad": 2, "precio_unitario": 2.0}
]
total = calcular_total(cesta_test)
print(f"✅ Total calculado: {total}€")

✅ Total calculado: 8.5€


# Apartado E

Crear una funcion que guarde el carrito de la compra en la BBDD

In [23]:
def guardar_carrito(username, carrito):
    """
    Guarda el carrito actualizado en Redis.
    """
    # Recalculo el importe total
    carrito['importe_total'] = calcular_total(carrito['cesta'])
    
    # Guardo en Redis
    r.set(f"carrito:{username}", json.dumps(carrito))
    print(f"Carrito guardado para '{username}'. Total: {carrito['importe_total']}€")

In [24]:
# Pruebo guardar
carrito = recuperar_carrito("maria_lopez")
carrito['cesta'].append({"articulo": "Leche", "cantidad": 2, "precio_unitario": 1.20})
guardar_carrito("maria_lopez", carrito)

Carrito recuperado para 'maria_lopez'
Carrito guardado para 'maria_lopez'. Total: 2.4€


# Apartado F

Crear una funcion que permita añadir un item al carrito

In [25]:
def añadir_item(username, item):
    """
    Añade un item al carrito.
    """
    carrito = recuperar_carrito(username)
    
    if carrito is None:
        carrito = crear_carrito(username)
    
    # Verifico si ya existe
    item_existente = False
    for i, existente in enumerate(carrito['cesta']):
        if existente['articulo'] == item['articulo']:
            carrito['cesta'][i]['cantidad'] += item['cantidad']
            item_existente = True
            print(f"Actualizada cantidad de '{item['articulo']}'")
            break
    
    if not item_existente:
        carrito['cesta'].append(item)
        print(f"Añadido '{item['articulo']}' al carrito")
    
    guardar_carrito(username, carrito)

In [26]:
# Pruebo añadir items
añadir_item("maria_lopez", {"articulo": "Yogur", "cantidad": 4, "precio_unitario": 0.80})
añadir_item("maria_lopez", {"articulo": "Huevos", "cantidad": 12, "precio_unitario": 0.25})

# Verifico
carrito = recuperar_carrito("maria_lopez")
print(f"\n✅ Items en el carrito: {len(carrito['cesta'])}")
print(f"✅ Total: {carrito['importe_total']}€")

Carrito recuperado para 'maria_lopez'
Añadido 'Yogur' al carrito
Carrito guardado para 'maria_lopez'. Total: 5.6€
Carrito recuperado para 'maria_lopez'
Añadido 'Huevos' al carrito
Carrito guardado para 'maria_lopez'. Total: 8.6€
Carrito recuperado para 'maria_lopez'

✅ Items en el carrito: 3
✅ Total: 8.6€


# Apartado G

Crear una funcion que permita eliminar un item del carrito

In [27]:
def eliminar_item(username, item_name):
    """
    Elimina un item del carrito.
    """
    carrito = recuperar_carrito(username)
    
    if carrito is None:
        print(f"No existe carrito para '{username}'")
        return
    
    # Elimino el item
    cesta_actualizada = [item for item in carrito['cesta'] if item['articulo'] != item_name]
    
    if len(cesta_actualizada) < len(carrito['cesta']):
        carrito['cesta'] = cesta_actualizada
        print(f"Eliminado '{item_name}' del carrito")
        guardar_carrito(username, carrito)
    else:
        print(f"No se encontró '{item_name}'")

In [28]:
# Pruebo eliminar
eliminar_item("maria_lopez", "Leche")

# Verifico
carrito = recuperar_carrito("maria_lopez")
print(f"✅ Items restantes: {len(carrito['cesta'])}")

Carrito recuperado para 'maria_lopez'
Eliminado 'Leche' del carrito
Carrito guardado para 'maria_lopez'. Total: 6.2€
Carrito recuperado para 'maria_lopez'
✅ Items restantes: 2


In [39]:
# Pruebo eliminar un producto del carrito
eliminar_item("maria_lopez", "Leche")

# Verifico el carrito actualizado
carrito = recuperar_carrito("maria_lopez")
print(f"\n✅ Productos restantes en el carrito: {len(carrito['cesta'])}")
print("\nDetalle de productos:")
for item in carrito['cesta']:
    print(f"  - {item['articulo']}: {item['cantidad']} unidades x {item['precio_unitario']}€")
print(f"\n💰 Total actualizado: {carrito['importe_total']}€")

Carrito recuperado para 'maria_lopez'
No se encontró 'Leche'
Carrito recuperado para 'maria_lopez'

✅ Productos restantes en el carrito: 0

Detalle de productos:

💰 Total actualizado: 0€


# Apartado H

Crear una funcion que permita vaciar completamente el carrito

In [36]:
def vaciar_carrito(username):
    """
    Vacía completamente el carrito de un usuario (simula una compra).
    
    Args:
        username (str): Nombre del usuario
    """
    # Recupero el carrito para mostrar información antes de vaciarlo
    carrito = recuperar_carrito(username)
    
    if carrito is None:
        print(f"No existe carrito para el usuario '{username}'")
        return
    
    total_compra = carrito['importe_total']
    num_items = len(carrito['cesta'])
    
    # Reinicio el carrito dejándolo vacío
    carrito['cesta'] = []
    carrito['importe_total'] = 0
    
    # Guardo el carrito vacío en Redis
    guardar_carrito(username, carrito)
    
    print(f"¡Compra realizada con éxito!")
    print(f"Total pagado: {total_compra}€ por {num_items} productos diferentes")

In [37]:
# Pruebo vaciar
vaciar_carrito("maria_lopez")

# Verifico que quedó vacío
carrito = recuperar_carrito("maria_lopez")
print(f"✅ Carrito vacío: {carrito}")

Carrito recuperado para 'maria_lopez'
Carrito guardado para 'maria_lopez'. Total: 0€
¡Compra realizada con éxito!
Total pagado: 0€ por 0 productos diferentes
Carrito recuperado para 'maria_lopez'
✅ Carrito vacío: {'username': 'maria_lopez', 'cesta': [], 'importe_total': 0}


In [38]:
# ========================================
# DEMOSTRACIÓN COMPLETA DEL SISTEMA
# ========================================

print("=" * 60)
print("SISTEMA DE GESTIÓN DE CARRITOS DE COMPRA - REDIS")
print("=" * 60)

# 1. Creo un nuevo usuario
print("\n1️⃣ Creación de carrito para nuevo usuario")
print("-" * 60)
crear_carrito("juan_perez")

# 2. Añado varios productos
print("\n2️⃣ Añadiendo productos al carrito")
print("-" * 60)
añadir_item("juan_perez", {"articulo": "Portátil Lenovo", "cantidad": 1, "precio_unitario": 899.99})
añadir_item("juan_perez", {"articulo": "Ratón inalámbrico", "cantidad": 2, "precio_unitario": 15.50})
añadir_item("juan_perez", {"articulo": "Teclado mecánico", "cantidad": 1, "precio_unitario": 75.00})
añadir_item("juan_perez", {"articulo": "Webcam HD", "cantidad": 1, "precio_unitario": 45.99})

# 3. Muestro el carrito completo
print("\n3️⃣ Estado actual del carrito")
print("-" * 60)
carrito = recuperar_carrito("juan_perez")
print(f"\nUsuario: {carrito['username']}")
print(f"Número de productos: {len(carrito['cesta'])}")
print("\nDetalle de productos:")
for item in carrito['cesta']:
    subtotal = item['cantidad'] * item['precio_unitario']
    print(f"  • {item['articulo']}: {item['cantidad']} x {item['precio_unitario']}€ = {subtotal}€")
print(f"\n💰 IMPORTE TOTAL: {carrito['importe_total']}€")

# 4. Elimino un producto
print("\n4️⃣ Eliminando un producto del carrito")
print("-" * 60)
eliminar_item("juan_perez", "Webcam HD")

# 5. Verifico el carrito actualizado
print("\n5️⃣ Carrito después de eliminar producto")
print("-" * 60)
carrito = recuperar_carrito("juan_perez")
print(f"Productos restantes: {len(carrito['cesta'])}")
print(f"💰 Nuevo total: {carrito['importe_total']}€")

# 6. Realizo la compra (vacío el carrito)
print("\n6️⃣ Realizando compra")
print("-" * 60)
vaciar_carrito("juan_perez")

# 7. Verifico que el carrito quedó vacío
print("\n7️⃣ Verificación final")
print("-" * 60)
carrito_final = recuperar_carrito("juan_perez")
print(f"Productos en carrito: {len(carrito_final['cesta'])}")
print(f"Importe total: {carrito_final['importe_total']}€")

print("\n" + "=" * 60)
print("✅ DEMOSTRACIÓN COMPLETADA EXITOSAMENTE")
print("=" * 60)

SISTEMA DE GESTIÓN DE CARRITOS DE COMPRA - REDIS

1️⃣ Creación de carrito para nuevo usuario
------------------------------------------------------------
Carrito creado para el usuario 'juan_perez'

2️⃣ Añadiendo productos al carrito
------------------------------------------------------------
Carrito recuperado para 'juan_perez'
Añadido 'Portátil Lenovo' al carrito
Carrito guardado para 'juan_perez'. Total: 899.99€
Carrito recuperado para 'juan_perez'
Añadido 'Ratón inalámbrico' al carrito
Carrito guardado para 'juan_perez'. Total: 930.99€
Carrito recuperado para 'juan_perez'
Añadido 'Teclado mecánico' al carrito
Carrito guardado para 'juan_perez'. Total: 1005.99€
Carrito recuperado para 'juan_perez'
Añadido 'Webcam HD' al carrito
Carrito guardado para 'juan_perez'. Total: 1051.98€

3️⃣ Estado actual del carrito
------------------------------------------------------------
Carrito recuperado para 'juan_perez'

Usuario: juan_perez
Número de productos: 4

Detalle de productos:
  • Portát